OCR'ing the documents was a huge challenge because of the noise in the documents. In this notebook, I try to -
* Preprocess the images to clean them before OCR'ing them.
* Postprocess badly OCR'ed documents.

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import words
from nltk.metrics.distance import (
    edit_distance,
    jaccard_distance,
    )
from nltk.util import ngrams
nltk.download('words')
import pandas
from PIL import Image
import numpy as np
import pytesseract
from autocorrect import Speller
import neuralcoref
spell = Speller()

[nltk_data] Downloading package words to
[nltk_data]     /Users/andrealphonse/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [39]:
import spacy
from spacy import displacy
from collections import Counter
nlp = spacy.load("en_core_web_lg")
path_to_patents = '/Volumes/Non-Backup_Files/US-patents/random_sample_seeded'

In [3]:
from patent_classification_utils import classify_page, classify_patent, extract_imgs, extract_text, page_has_drawing
from detectron2.data.detection_utils import read_image
from ocrutils import OCR_Pages
import os

In [40]:
# Trying out on example
# FIG_PATH = '/Volumes/Non-Backup_Files/US-patents/random_sample_seeded/430/00000001.tif'
# TEXT_PATH = '/Volumes/Non-Backup_Files/US-patents/random_sample_seeded/430/00000002.tif'
FIG_PATH = '/Volumes/Non-Backup_Files/US-patents/random_sample_seeded/604/00000001.tif'
TEXT_PATH = '/Volumes/Non-Backup_Files/US-patents/random_sample_seeded/1645/00000002.tif'#'/Volumes/Non-Backup_Files/US-patents/random_sample_seeded/604/00000002.tif'
fig_im = Image.open(FIG_PATH)
text_im = Image.open(TEXT_PATH)
fig_array = np.array(fig_im)
text_array = np.array(text_im)

In [44]:
text_im.convert("RGB").save('jpegtest.jpg', "JPEG", quality=100)

In [45]:
#easyocr attempt
import easyocr
reader = easyocr.Reader(['en'], gpu = False) # need to run only once to load model into memory
result = reader.readtext('jpegtest.jpg')

Using CPU. Note: This module is much faster with a GPU.

KeyboardInterrupt



In [5]:
# img0 = read_image(TEXT_PATH, format="BGR")
# img1 = read_image(FIG_PATH, format="BGR")
# a = extract_text([img0, img1])
# # outputs = classify_patent([img0, img1])

In [6]:
# from matplotlib import pyplot as plt
# # change the figure size
# fig = plt.figure(figsize = (20,40)) # create a 5 x 5 figure 
# ax = fig.add_subplot(111)
# ax.imshow(a[1], interpolation='none')
# ax.set_title('larger figure')

# plt.show()

In [7]:
tifs = []
for path, dirs, files in os.walk("{}/".format(path_to_patents)):
    for f in files:
        if f.endswith('tif'):
            tifs.append('{}/{}'.format(path, f))

In [8]:
len(tifs)

8205

In [37]:
from scipy import ndimage, misc
#digitization
def ocr_median_filter(patent_nb):
    """
    function to ocr a given patent with tesseract after applying a median filter to clean the images
    Args:
    patent_nb - the patent number
    Returns
    the OCR of the patent after applying median filtering to the pages
    """
    #fetch all paths to tif's
    tifs = []
    for path, dirs, files in os.walk("{}/{}/".format(path_to_patents, patent_nb)):
        for f in files:
            if f.endswith('tif'):
                tifs.append('{}/{}'.format(path, f))
    tif_imgs = [read_image(tif, format="BGR") for tif in tifs]
    text_indices = []
    for i in range(len(tif_imgs)):
        if not page_has_drawing(tif_imgs[i]):
            text_indices.append(i)
    return OCR_Pages([ndimage.median_filter(np.array(Image.open(tifs[i])), size=4) for i in text_indices])

In [38]:
example = ocr_median_filter(1645)
print(example)

Config '/Users/andrealphonse/Documents/UniStuff/MA/MA3/Classes/Patent Project/PubLayNet/detectron2/detectron_config/DLA_mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Config '/Users/andrealphonse/Documents/UniStuff/MA/MA3/Classes/Patent Project/PubLayNet/detectron2/detectron_config/DLA_mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Config '/Users/andrealphonse/Documents/UniStuff/MA/MA3/Classes/Patent Project/PubLayNet/detectron2/detectron_config/DLA_mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Config '/Users/andrealphonse/Documents/UniStuff/MA/MA3/Classes/Patent Project/PubLayNet/detectron2/detectron_config/DLA_mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


UNITED StTaTES PATENT OFFICE.

SETH LAMB, OF NEW YORK, N.Y.

IMPROVEMENT IN MOWING-MACHINES.

Specification forming part of Letters Patent No. 1.685, dated Jine 20, 1°40,

To allachom it may concern:

Be it known that 1,Serm LAMB, of the city,
county, and Stateof New York, have invented,
coustructed, and applied to use a new and use-
ful Improvement in the Machinery for Cutting
Cir. sand Grain; and I do hereby declare that
the sollowing is a fall and exact description
thereof, with reference to the accompanying
drawings, making part of this specitication,
and in which the same parts are designated
by the same letters of reference in all the dif-
ferent figures.

Figure 1 represents a vertical projection of
the geared side of the machine; Tig. 2, a top
view, witha part of the floor or casing removed
to exhibit the sickle-wheel, gearing, and pul-
leys and to exhibit their connection and posi-

 

tion with the frame of the machine; Fig. 3, a,

front vertical projection.
tive view of the 

In [30]:
lines = example[178:].split('\n')
lines

['To all whom tt may concern:',
 '',
 'Be it known that I,SerH Lams, of the city,',
 'county,and State of New York, have invented,',
 'constructed, and applied to use a new and use-',
 '',
 '‘ful Improvement in the Machinery for Cutting',
 '',
 'Gro sd4ndGrain; andI do hereby declare that',
 'the rollowing is a fall and exact description',
 'thereof, with reference to the accompanying',
 'drawings, making part of this specification,',
 'and in which the same parts are designated',
 'by the same letters of reference in all the dif-',
 'ferent figures.',
 '',
 'Figure 1 represents a vertical projection of',
 'the geared side of the machine; Fig. 2, a top',
 'view, witha part of the floor or casing removed',
 'to exhibit the sickle-wheel, gearing, and pal-',
 'leys and to exhibit their connection and posi-',
 'tion with the frame of the machine; Fig. 3, a',
 'front vertical projection. Tig. 4 is a perspec-',
 'tive view of the sickle guides or rests, with bows',
 'or guards extending from

In [ ]:
#attempt at correcting bad ocr result, did not work as bad result is very unpredictable
revamped_str = ''
lines_to_append = []
for line in lines:
    if '|' in line:
        split = line.split('|')
        revamped_str += split[0]
        lines_to_append.append(split[1])
    else:
        if lines_to_append

In [ ]:
# attempt to use NLP to detect claims

In [20]:
example.lower().find('to all whom')

178

In [25]:
example[:178], re.sub("\s\s+", " ", example[178:])

('~Unitep StratTes PATENT OFFICE.\n\nSETH LAMB, OF NEW YORK, N. Y.\n\nIMPROVEMENT IN MOWING-MACHINES.\n\n \n\nSpecification forming part of Letters Patent No. 1,645, dated June 20, 1840,\n\n',
 'To all whom tt may concern: Be it known that I,SerH Lams, of the city,\ncounty,and State of New York, have invented,\nconstructed, and applied to use a new and use- ‘ful Improvement in the Machinery for Cutting Gro sd4ndGrain; andI do hereby declare that\nthe rollowing is a fall and exact description\nthereof, with reference to the accompanying\ndrawings, making part of this specification,\nand in which the same parts are designated\nby the same letters of reference in all the dif-\nferent figures. Figure 1 represents a vertical projection of\nthe geared side of the machine; Fig. 2, a top\nview, witha part of the floor or casing removed\nto exhibit the sickle-wheel, gearing, and pal-\nleys and to exhibit their connection and posi-\ntion with the frame of the machine; Fig. 3, a\nfront vertical p

In [26]:
import re
print(example[:178], re.sub("\s\s+", " ", example[178:].replace('\n',' ')))

~Unitep StratTes PATENT OFFICE.

SETH LAMB, OF NEW YORK, N. Y.

IMPROVEMENT IN MOWING-MACHINES.

 

Specification forming part of Letters Patent No. 1,645, dated June 20, 1840,

 To all whom tt may concern: Be it known that I,SerH Lams, of the city, county,and State of New York, have invented, constructed, and applied to use a new and use- ‘ful Improvement in the Machinery for Cutting Gro sd4ndGrain; andI do hereby declare that the rollowing is a fall and exact description thereof, with reference to the accompanying drawings, making part of this specification, and in which the same parts are designated by the same letters of reference in all the dif- ferent figures. Figure 1 represents a vertical projection of the geared side of the machine; Fig. 2, a top view, witha part of the floor or casing removed to exhibit the sickle-wheel, gearing, and pal- leys and to exhibit their connection and posi- tion with the frame of the machine; Fig. 3, a front vertical projection. Tig. 4 is a perspec

In [14]:
print(example.strip())

~Unitep StratTes PATENT OFFICE.

SETH LAMB, OF NEW YORK, N. Y.

IMPROVEMENT IN MOWING-MACHINES.

 

Specification forming part of Letters Patent No. 1,645, dated June 20, 1840,

To all whom tt may concern:

Be it known that I,SerH Lams, of the city,
county,and State of New York, have invented,
constructed, and applied to use a new and use-

‘ful Improvement in the Machinery for Cutting

Gro sd4ndGrain; andI do hereby declare that
the rollowing is a fall and exact description
thereof, with reference to the accompanying
drawings, making part of this specification,
and in which the same parts are designated
by the same letters of reference in all the dif-
ferent figures.

Figure 1 represents a vertical projection of
the geared side of the machine; Fig. 2, a top
view, witha part of the floor or casing removed
to exhibit the sickle-wheel, gearing, and pal-
leys and to exhibit their connection and posi-
tion with the frame of the machine; Fig. 3, a
front vertical projection. Tig. 4 is a pers

In [11]:
patent_str_corr = spell(example)

In [12]:
print(patent_str_corr)

~United States PATENT OFFICE.

SET LAM, Of NEW OR, N. Y.

IMPROVEMENT In MOWING-MACHINES.

 

Specification forming part of Letters Patent No. 1,645, dated June 20, 1840,

To all whom tt may concern:

Be it known that I,Ser Laws, of the city,
county,and State of New York, have invented,
constructed, and applied to use a new and use-

‘ful Improvement in the Machinery for Cutting

Gro sd4drain; and do hereby declare that
the following is a fall and exact description
thereof, with reference to the accompanying
drawings, making part of this specification,
and in which the same parts are designated
by the same letters of reference in all the dif-
fervent figures.

Figure 1 represents a vertical projection of
the geared side of the machine; Fig. 2, a top
view, with part of the floor or casing removed
to exhibit the sickle-wheel, gearing, and pal-
less and to exhibit their connection and pos-
tion with the frame of the machine; Fig. 3, a
front vertical projection. Big. 4 is a perspec-
time v

In [22]:
#claim extraction attempt (with coref module to detect action of claiming)
neuralcoref.add_to_pipe(nlp)
example_doc = nlp(example)

In [39]:
index = 0
nounIndices = []
for token in example_doc:
    # print(token.text, token.pos_, token.dep_, token.head.text)
    if token.pos_ == 'PRON':
    # if token.pos_ == 'VERB':
        
        nounIndices.append(index)
    index = index + 1

In [68]:
for pronoun in nounIndices:
#     if(example_doc[pronoun].text.lower() == 'i'):
    head_index = example_doc[pronoun].head.i
    print(example_doc[pronoun].text, ': ', example_doc[head_index-5:head_index+15].text + '\n')

whom :  To all whom it may concern:

Be it known that I, With1As T, SPROUSE,


it :  To all whom it may concern:

Be it known that I, With1As T, SPROUSE,


it :  concern:

Be it known that I, With1As T, SPROUSE,
of the county of Sangamon

I :  
- ois, have discovered a new and useful Im-
movement in the Manufacture of Flows; and

i :  and
i do hereby declare that the following is a full
and exact description of said improvement and

them :  irons as of stock-
ing them, the mold-board and bar being made
out of a

i :  
forthebar. Givethelarger—i. e. the diamond-
shaped—part of the plate a gradual bend

What :  # staple.

What I, the said WILLIAM .T. SPROUSE,
claim as my own invention,

I :  

What I, the said WILLIAM .T. SPROUSE,
claim as my own invention, and not previously

them :  and bending
instead of making them out of two pieces of -
iron and welding time together.



In [73]:
example_doc[nounIndices[-2]].head.head

said

In [91]:
for entity in example_doc.ents:
    if entity.label_ == 'PERSON':
        print(entity[0].head.head.head.head)

SPROUSE
seth
fastened
attached
WILLIAM
SPROUSE
HMAS
BARKINSON


In [40]:
index = 0
verbIndices = []
for token in example_doc:
    # print(token.text, token.pos_, token.dep_, token.head.text)
#     if token.pos_ == 'PRON':
    if token.pos_ == 'VERB':
        
        verbIndices.append(index)
    index = index + 1

In [34]:
[example_doc[i] for i in verbIndices]

[forming,
 dated,
 may,
 known,
 discovered,
 declare,
 following,
 said,
 making,
 called,
 ’s,
 differs,
 making,
 ing,
 made,
 Take,
 say,
 Draw,
 said,
 said,
 said,
 leaving,
 said,
 said,
 Cut,
 said,
 Make,
 said,
 said,
 Bend,
 said,
 cutting,
 shaped,
 completed,
 made,
 made,
 stocked,
 placed,
 attached,
 passing,
 directed,
 made,
 said,
 extended,
 fastened,
 extending,
 and.attaching,
 fastened,
 attached,
 said,
 claim,
 known,
 described,
 cutting,
 bending,
 making]

In [37]:
for verb in verbIndices:
    if(example_doc[verb].text == 'claim'):
        print(example_doc[verb])

claim


In [26]:
example_doc._.coref_clusters

[it: [it, it],
 the plate: [the plate, the plate, the plate],
 the seth and fastened by
 a tap, and that through the mold extending
 also through and.attaching the left-hand han-
 dle: [the seth and fastened by
 a tap, and that through the mold extending
 also through and.attaching the left-hand han-
 dle, the seth, aud],
 SPROUSE,
 : [SPROUSE,
 , them]]

In [27]:
#named entity recognition
for entity in example_doc.ents:
    print(entity.label_, ' | ', entity.text)

GPE  |  PaTENT
PERSON  |  WILLIAM T. SPROUSE
GPE  |  SANGAMON
GPE  |  ILLINOIS
ORG  |  PLOS
CARDINAL  |  604
DATE  |  February 15, 1838
ORG  |  With1As T
ORG  |  SPROUSE
GPE  |  Sangamon
ORG  |  State
ORG  |  the Manufacture of Flows
CARDINAL  |  twelve
CARDINAL  |  12
CARDINAL  |  eighteen
QUANTITY  |  18)
inches
CARDINAL  |  twelve
CARDINAL  |  12
CARDINAL  |  six
QUANTITY  |  6) inches
CARDINAL  |  twelve
CARDINAL  |  12
CARDINAL  |  one
QUANTITY  |  a half-square
CARDINAL  |  one-half
QUANTITY  |  a half-inch
CARDINAL  |  about one and a half
QUANTITY  |  a half-square
CARDINAL  |  two
PERSON  |  seth
ORG  |  and.attaching
PERSON  |  seth
PERSON  |  seth
CARDINAL  |  #
PERSON  |  WILLIAM .T.
CARDINAL  |  two
GPE  |  Springfield
DATE  |  November 30, 1836
PERSON  |  WILLIAM T. SPROUSE
PERSON  |  B. HMAS
PERSON  |  Jr.


In [ ]:
import nltk
text = nltk.word_tokenize("And now for something completely different")
nltk.pos_tag(text)

In [16]:
with open("/Volumes/Non-Backup_Files/US-patents/random_sample_seeded_txt/17090.txt", "r") as f:
    patent = f.read()

In [17]:
print(patent)


J. 0. HALSEY,

Gas Retort.

Patented April 21, 1857,

_ No.17,090,

 

 

 

 

 

 

 

 

 

 

 

 

10

15

- 20

25

30

35

40

45

UNITED STATES PATENT OFFICE.

JAMES 0. HALSEY, OF ESSEX, NEW JERSHY.

PORTABLE APPARATUS FOR GENERATING ILLUMINATING-GAS.

Specification of Letters Patent No. 17,090, dated April 21, 1857,

To ali whom vé may concern:

Be it known that I, James O. Hatsry,
of the county of Essex, State of New. Jer-
sey, have invented a new and Improved
Portable Gas-Generator; and I do hereby
declare that the following is a full and ex-
act description of the same, reference being
had to the accompanying drawings, and to
the letters of reference marked thereon, in
which— ‘

Figure 1 isa perspective and Figs. 2 and
3 are sectional views.

Similar letters of reference in the several
figures denote the same parts of the appa-
ratus.

The nature of my invention consists in
combining an air chamber with a stove and
retort for generating gas, for the purpose
of allowing sa

In [19]:
# autocorrect
corrected_patent = spell(patent)
print(corrected_patent)


J. 0. HALSEY,

Gas Report.

Patented April 21, 1857,

_ No.17,090,

 

 

 

 

 

 

 

 

 

 

 

 

10

15

- 20

25

30

35

40

45

UNITED STATES PATENT OFFICE.

AES 0. HALSEY, Of SSE, NEW JERSHY.

PORTAL APPARATUS OR GENERATING ILLUMINATING-AS.

Specification of Letters Patent No. 17,090, dated April 21, 1857,

To ali whom vé may concern:

Be it known that I, James O. Harry,
of the county of Essex, State of New. Jer-
see, have invented a new and Improved
Portable Gas-Generator; and I do hereby
declare that the following is a full and ex-
act description of the same, reference being
had to the accompanying drawings, and to
the letters of reference marked thereon, in
which— ‘

Figure 1 isa perspective and Pigs. 2 and
3 are sectional views.

Similar letters of reference in the several
figures denote the same parts of the apps-
rates.

The nature of my invention consists in
combining an air chamber with a stove and
report for generating gas, for the purpose
of allowing said report

In [20]:
patent_nlp = nlp(patent)

In [21]:
len(patent_nlp.ents)

28

In [8]:
labels = [x.label_ for x in patent_nlp.ents]
Counter(labels)

Counter({'ORG': 4,
         'DATE': 3,
         'CARDINAL': 7,
         'PERSON': 9,
         'GPE': 2,
         'PRODUCT': 2,
         'PERCENT': 1})

In [12]:
items = [x.text for x in patent_nlp.ents]
Counter(items).most_common(3)

[('HALSEY', 3), ('April 21, 1857', 2), ('one', 2)]

In [13]:
items

['HALSEY',
 'April 21, 1857',
 '10',
 '35\n\n',
 'UNITED STATES PATENT OFFICE',
 'JAMES 0',
 'HALSEY',
 'ESSEX',
 '17,090',
 'April 21, 1857',
 'ali',
 'vé',
 'James O. Hatsry',
 'Essex',
 'State of New',
 'Improved\nPortable',
 '1',
 '2',
 'one',
 'S',
 'one',
 'pense',
 'Letters Patent',
 'HALSEY',
 'McConxey',
 'E. C. Morea',
 '50\n\n60%',
 '70']